In [1]:
!apt-get install openjdk-11-jdk -y
!wget https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xzf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install sodapy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 34 not upgraded.
Need to get 5,366 kB of archives.
After this operation, 15.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jam

In [2]:
from sodapy import Socrata
import pandas as pd
import os
import findspark
import json
from google.colab import drive
from pyspark.sql import SparkSession
import pandas as pd


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"


findspark.init()
drive.mount('/content/drive')
spark = SparkSession.builder.appName("Colab Spark").getOrCreate()
spark

Mounted at /content/drive


In [71]:
housing_path = "/content/drive/My Drive/big_data/project/Housing_Maintenance_Code_Complaints_and_Problems_20250427.csv"
fire_path="/content/drive/My Drive/big_data/project/Fire_Incident_Dispatch_Data_20250429.csv"

In [72]:
from pyspark.sql.functions import min,split, col, concat_ws,regexp_replace, trim, upper,when,pandas_udf,to_date
import pandas as pd
import requests
from pyspark.sql.types import DoubleType, StructType, StructField

In [76]:
# Start SparkSession
spark = SparkSession.builder.appName("Housing Maintenance Code Complaints").getOrCreate()

# Read the CSV
original_housing_df = spark.read.option("header", True).csv(housing_path)
original_fire_df = spark.read.option("header", True).csv(fire_path)

housing_df = original_housing_df.na.drop()
housing_df.show(5,truncate=False)
# last_row = housing_df.tail(1)[0]
# print(last_row)
# housing_row_count = housing_df.count()
# print(f"Total rows: {housing_row_count}")

# print()

original_fire_df.show(5,truncate=False)
# fire_last_row = fire_df.tail(1)[0]
# print(fire_last_row)
# fire_row_count = fire_df.count()
# print(f"Total rows: {fire_row_count}")

o1fire_df = original_fire_df.na.drop()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

filtered_fire_df = o1fire_df.filter(
    col("INCIDENT_CLASSIFICATION").contains("Carbon Monoxide") |
    col("INCIDENT_CLASSIFICATION").contains("Fire")
)

housing_df = housing_df.withColumn("Received Date", to_date("Received Date", "MM/dd/yyyy"))
fire_df = filtered_fire_df.withColumn("INCIDENT_DATETIME", to_date("INCIDENT_DATETIME", "MM/dd/yyyy"))
start_date = "2023-01-01"
end_date = "2025-03-07"
filtered_housing_df = housing_df.filter(
    (col("Received Date") >= start_date) & (col("Received Date") <= end_date)
)

# # Filter fire incidents
filtered_fire_df = fire_df.filter(
    (col("INCIDENT_DATETIME") >= start_date) & (col("INCIDENT_DATETIME") <= end_date)
)

filtered_fire_df.show(truncate=False)

print(f"Filtered housing rows: {filtered_housing_df.count()}")
print(f"Filtered fire rows: {filtered_fire_df.count()}")
filtered_housing_df.show()

+-------------+----------+------------+-----------+--------+------------+----------------+---------+-----+---+---------+---------------+-------------+----------------+---------+--------------+---------------+------------------------+----------------+---------------------+--------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+------------------------+----------+------------------+-------------------+----------------+------------+-------+----------+----------------------------------------------+
|Received Date|Problem ID|Complaint ID|Building ID|Borough |House Number|Street Name     |Post Code|Block|Lot|Apartment|Community Bo

In [6]:
house_selected_df = housing_df.select("Received Date", "Borough","Street Name","Post Code","Problem Status","Latitude","Longitude")
house_selected_df.show()
fire_selected_df=fire_df.select("INCIDENT_DATETIME","ALARM_BOX_BOROUGH","ALARM_BOX_LOCATION","INCIDENT_BOROUGH","ZIPCODE")
fire_selected_df.show(truncate=False)

+-------------+---------+-----------------+---------+--------------+------------------+-------------------+
|Received Date|  Borough|      Street Name|Post Code|Problem Status|          Latitude|          Longitude|
+-------------+---------+-----------------+---------+--------------+------------------+-------------------+
|   2018-01-03| BROOKLYN|  ST MARKS AVENUE|    11213|         CLOSE|40.674022999999998|-73.928471999999999|
|   2017-01-08|    BRONX|  PROSPECT AVENUE|    10457|         CLOSE|         40.848554|-73.885430999999997|
|   2018-01-26| BROOKLYN| HOMECREST AVENUE|    11235|         CLOSE|40.584297999999997|-73.956289999999996|
|   2017-11-01|    BRONX|    DAWSON STREET|    10459|         CLOSE|40.819769999999998|-73.897623999999993|
|   2017-11-01| BROOKLYN|     CROWN STREET|    11225|         CLOSE|40.666344000000002|-73.952019000000007|
|   2017-11-09|    BRONX|  ROSEDALE AVENUE|    10473|         CLOSE|40.815274000000002|-73.865380000000002|
|   2017-11-09|MANHATTAN|   

In [7]:
fire_address_df = fire_selected_df.withColumn("normalized_location", upper(col("ALARM_BOX_LOCATION")))

# Replace all separators (OPP, AND, /, @, AT) with "&"
fire_address_df = fire_address_df.withColumn(
    "normalized_location",
    regexp_replace(col("normalized_location"), r"\b(OPP|AND|/|@|AT)\b", "&")
)

# Remove compass directions  and other noise words (TO, BET, BETWEEN, HOSPITAL, AIRPORT)
fire_address_df = fire_address_df.withColumn("normalized_location",  regexp_replace(       col("ALARM_BOX_LOCATION"), r"^(?:[NSEW]{1,2}/[NSEW]{1,2}|[NSEW]{1,2})\s+", ""))

fire_address_df = fire_address_df.withColumn(
    "normalized_location",
    regexp_replace(col("normalized_location"), r"\b(NORTH|SOUTH|EAST|WEST|TO|BET|BETWEEN|HOSPITAL|AIRPORT|OF)\b", "")
)

# Split by "&" and clean up
fire_address_df= fire_address_df.withColumn("address_parts", split(col("normalized_location"), "&")) \
       .withColumn("address_1", trim(col("address_parts")[0])) \
       .withColumn("address_2", trim(col("address_parts")[1]))

# convert borough to borough code for API
fire_address_df = fire_address_df.withColumn(
    "borough_code",
    when(col("INCIDENT_BOROUGH") == "MANHATTAN", 1)\
    .when(col("INCIDENT_BOROUGH") == "BRONX", 2)\
    .when(col("INCIDENT_BOROUGH") == "BROOKLYN", 3)\
    .when(col("INCIDENT_BOROUGH") == "QUEENS", 4)\
    .when(col("INCIDENT_BOROUGH").startswith("RICHMOND"), 5)
    .otherwise(None)
)

# fire_address_df.show(n=40, truncate=False)
null_address2_df = fire_address_df.filter(col("address_2").isNull())
# print(null_address2_df) #total 117095
null_address2_df.show(truncate=False)

+-----------------+------------------------+---------------------------------------+------------------------+-------+-------------------------------------+---------------------------------------+-------------------------------------+---------+------------+
|INCIDENT_DATETIME|ALARM_BOX_BOROUGH       |ALARM_BOX_LOCATION                     |INCIDENT_BOROUGH        |ZIPCODE|normalized_location                  |address_parts                          |address_1                            |address_2|borough_code|
+-----------------+------------------------+---------------------------------------+------------------------+-------+-------------------------------------+---------------------------------------+-------------------------------------+---------+------------+
|2025-03-07       |MANHATTAN               |WARDS ISL DEPT HOMLESS SERVICES        |MANHATTAN               |10035  |WARDS ISL DEPT HOMLESS SERVICES      |[WARDS ISL DEPT HOMLESS SERVICES]      |WARDS ISL DEPT HOMLESS SERVICES   

In [24]:
sample_df = fire_address_df.limit(100)
rows = sample_df.select("borough_code", "address_1", "address_2").collect()
results = []

In [59]:
def get_lat_lon_from_streets(borough_code, street1, street2):
    url = "https://geoservice.planning.nyc.gov/geoservice/geoservice.svc/Function_2"
    params = {
        "Borough1": borough_code,
        "Street1": street1,
        "Borough2": borough_code,
        "Street2": street2,
        "DisplayFormat": "True",
        "Key": "5u8xLAUDBG3KbPeS"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        try:
            lat = data["display"].get("out_latitude", "").strip()
            lon = data["display"].get("out_longitude", "").strip()
            if lat and lon:
                return float(lat), float(lon)
        except Exception as e:
            print(f"Parsing error: {e}")

    return None, None


In [66]:
cache = {}
results = []

for row in rows:
    key = (row['borough_code'], row['address_1'], row['address_2'])

    if key in cache:
        lat, lon = cache[key]
    else:
        lat, lon = get_lat_lon_from_streets(*key)
        cache[key] = (lat, lon)

    results.append({
        "borough_code": key[0],
        "street1": key[1],
        "street2": key[2],
        "latitude": lat,
        "longitude": lon
    })

result_spark_df = spark.createDataFrame(results)
filtered_spark_df = result_spark_df.filter(
    (result_spark_df.latitude.isNotNull()) & (result_spark_df.longitude.isNotNull())
)
print(f"Number of valid coordinate rows: {filtered_spark_df.count()}")
filtered_spark_df.show(20, truncate=False)

Number of valid coordinate rows: 79
+------------+---------+----------+--------------+------------+
|borough_code|latitude |longitude |street1       |street2     |
+------------+---------+----------+--------------+------------+
|3           |40.690974|-73.94833 |MARCY AVE     |KOSCIUSKO ST|
|1           |40.818236|-73.952722|AMSTERDAM AVE |W 134 ST    |
|1           |40.79037 |-73.949766|PARK AVE      |E 102 ST    |
|4           |40.676598|-73.795935|120 AVE       |144 ST      |
|1           |40.753909|-73.999614|10 AVE        |W 33 ST     |
|4           |40.687904|-73.849699|ATLANTIC AVE  |93 ST       |
|4           |40.709458|-73.739898|109 AVE       |217 PL      |
|3           |40.701439|-73.984762|YORK ST       |BRIDGE ST   |
|3           |40.629435|-73.967075|AVENUE H      |E 10 ST     |
|1           |40.745826|-74.001812|9 AVE         |W 22 ST     |
|3           |40.702192|-73.928337|WILSON AVE    |JEFFERSON ST|
|3           |40.670118|-73.922467|RALPH AVE     |ST JOHNS PL |
|4  